In [1]:
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import cv2
import os
import random
import secrets

In [2]:
def encrypt_string(key, input_string):
    # Convert the key and input string to bytes
    key_bytes = bytes(key, 'utf-8')
    input_bytes = bytes(input_string, 'utf-8')
    # Pad the input bytes to a multiple of 16 bytes
    padded_input_bytes = pad(input_bytes, AES.block_size)
    # Generate an AES cipher object with CBC mode and the provided key
    cipher = AES.new(key_bytes, AES.MODE_CBC)
    # Encrypt the padded input bytes
    encrypted_bytes = cipher.encrypt(padded_input_bytes)
    # Combine the encrypted bytes and the initialization vector (IV)
    iv_and_encrypted_bytes = cipher.iv + encrypted_bytes
    # Convert the combined bytes to a base64-encoded string
    encrypted_string = base64.b64encode(iv_and_encrypted_bytes).decode('utf-8')
    return encrypted_string

In [3]:
def steganography_image(password, message, input_image_path, output_image_path):
    # Load the image
    img = cv2.imread(input_image_path)
    height, width, channels = img.shape
    index = 0
    max_index = len(message)
    
    # Convert the message to binary
    binary_message = ''.join(format(ord(i), '08b') for i in message)

    # Loop through each pixel and hide the message in the least significant bit
    for y in range(height):
        for x in range(width):
            r, g, b = img[y, x]
            if index < max_index:
                # Extract the least significant bit of each channel
                r_bit = r & 1
                g_bit = g & 1
                b_bit = b & 1
                
                # Get the current bit of the binary message
                current_bit = int(binary_message[index])
                
                # Hide the bit in the least significant bit of each channel
                r = (r & ~1) | current_bit
                index += 1
                if index < max_index:
                    g = (g & ~1) | int(binary_message[index])
                    index += 1
                if index < max_index:
                    b = (b & ~1) | int(binary_message[index])
                    index += 1
                
                # Update the pixel with the modified channels
                img[y, x] = (r, g, b)
            else:
                # If the entire message has been hidden, break out of the loop
                break

    # Save the modified image to disk
    cv2.imwrite(output_image_path, img)

In [17]:
def decrypt_string(key, encrypted_string):
    # Convert the key and encrypted string to bytes
    key_bytes = bytes(key, 'utf-8')
    encrypted_bytes = base64.b64decode(encrypted_string)
    # Extract the IV from the first 16 bytes of the encrypted bytes
    iv = encrypted_bytes[:16]
    # Generate an AES cipher object with CBC mode and the provided key and IV
    cipher = AES.new(key_bytes, AES.MODE_CBC, iv)
    # Decrypt the remaining encrypted bytes
    decrypted_bytes = cipher.decrypt(encrypted_bytes[16:])
    # Unpad the decrypted bytes and return as bytes object
    unpadded_decrypted_bytes = unpad(decrypted_bytes, AES.block_size)
    return unpadded_decrypted_bytes

In [18]:
def desteganography_image(input_image_path):
    # Load the image
    img = cv2.imread(input_image_path)
    height, width, channels = img.shape
    index = 0
    message = ''
    
    # Loop through each pixel and extract the least significant bit
    for y in range(height):
        for x in range(width):
            r, g, b = img[y, x]
            
            # Extract the least significant bit of each channel
            r_bit = r & 1
            g_bit = g & 1
            b_bit = b & 1
            
            # Add the bits to the message
            message += str(r_bit)
            index += 1
            if index < height * width:
                message += str(g_bit)
                index += 1
            if index < height * width:
                message += str(b_bit)
                index += 1
                
    # Convert the binary message to ASCII characters
    chars = [message[i:i+8] for i in range(0, len(message), 8)]
    ascii_message = ''.join([chr(int(c, 2)) for c in chars])

    return ascii_message

In [19]:
print("************************Encryption*******************************")
password = "abc@1234"
sentotp = "1HDF36%"
username = "maverick"
ip_path = r"C:\Users\maver\OneDrive\Documents\Giridhar\sem6\ISM - CSE3502\JCOMP_NEW\onlinetransaction\tostegimages"
key = secrets.token_hex(16)
img = random.randint(1,6)
ip_file = os.path.join(ip_path,str(img)+'.jpg')
text = password+sentotp
op_path = r"C:\Users\maver\OneDrive\Documents\Giridhar\sem6\ISM - CSE3502\JCOMP_NEW\onlinetransaction\opsteg"
op_file = os.path.join(op_path,username+'.jpg')

enc = encrypt_string(key,text)
print(enc)
steganography_image('ismbank', text, ip_file, op_file)
print('image has been steganographed')
print("************************Decryption*******************************")

data = decrypt_string(key,enc)
print('Decrypted data:',data)

************************Encryption*******************************
MR2fQIGWpwJvscanSp8wCgeOFvp9H3if56SdOKjf1MM=
image has been steganographed
************************Decryption*******************************
Decrypted data: b'abc@12341HDF36%'


In [20]:
key = '0123456789abcdef'
input_string = 'This is a secret message'
encrypted_string = encrypt_string(key, input_string)
print('Encrypted string:', encrypted_string)

# Decrypt the encrypted string
decrypted_string = decrypt_string(key, encrypted_string)
print('Decrypted string:', decrypted_string)

Encrypted string: 3SXUBWUE7LYsUz5ePgXivtMuvkkplqG/O/ff9iO80bEAJuVDuktfxxhWxwuqyWKL
Decrypted string: b'This is a secret message'
